In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle
import xgboost as xgb


def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [8]:

features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']

In [16]:
x_training_data=recession_data[recession_data['Date'] <= '2018-04-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-04-01']


In [9]:
X_train, X_test, y_train, y_test= train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.20)

In [53]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(random_state=88)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train.iloc[:,1:7], y_train['Recession_in_24mo'])

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
models = [{'name': 'lg','label': 'Logistic Regression',
           'classifier': LogisticRegression(random_state=88),
           'grid': {"C":np.logspace(-3,3,7), "penalty":["l2"]}},
          
          {'name': 'knn','label':'K Nearest Neighbors',
           'classifier':KNeighborsClassifier(),
           'grid': {"n_neighbors":np.arange(8)+1}},
          {
              'name':'xg','label':'xgboost','classifier':xgb.XGBClassifier(objective ='binary:logistic', learning_rate = 0.1,
                 booster='gbtree',alpha = 10, n_estimators = 10),'grid':{
                  'max_depth':range(3,10,1),
                'gamma':[i/10.0 for i in range(0,5)],
                 'colsample_bytree':[i/10.0 for i in range(1,10)],
                 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05] 
              }
          },
          
          {'name': 'dt','label': 'Descision Tree', 
           'classifier': DecisionTreeClassifier(random_state=88),
           'grid': {"max_depth":np.arange(8)+1}},
          {'name': 'sv', 'label': 'SVC (RBF)',
           'classifier':SVC(random_state=88),
           'grid': {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}}]

In [70]:
from sklearn.metrics import roc_auc_score
def model_selection(classifier, name, grid, X_train, y_train, scoring,i):
    
    gridsearch_cv=GridSearchCV(classifier, 
                               grid,
                               cv=5, 
                               scoring = scoring,verbose=2,n_jobs=4)
    
    gridsearch_cv.fit(X_adasyn, y_adasyn)
    
    results_dict = {}
    
    results_dict['classifier_name'] = name    
    results_dict['classifier'] = gridsearch_cv.best_estimator_
    results_dict['best_params'] = gridsearch_cv.best_params_
    results_dict['ROC_AUC'] = gridsearch_cv.best_score_
    filename = i+'.'+name
    pickle.dump(gridsearch_cv.best_estimator_, open(filename, 'wb'))
    
    return(results_dict)
results = []


In [71]:
for i in output_feature:
    X_adasyn, y_adasyn = adasyn.fit_resample(X_train.iloc[:,1:7], y_train[i])
    for m in models:  
        print(m['name'])    
        results.append(model_selection(m['classifier'], 
                                   m['name'],
                                   m['grid'],
                                   X_adasyn, 
                                   y_adasyn, 
                                   'roc_auc',i))      
        print('completed')
    results_df = pd.DataFrame(results).sort_values(by='ROC_AUC', ascending = False)
    results_df.to_csv(i+'.csv')
    
    
    








lg
Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  28 out of  35 | elapsed:    3.5s remaining:    0.9s
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:    3.5s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


completed
knn
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


completed
xg
Fitting 5 folds for each of 1575 candidates, totalling 7875 fits


...

[Parallel(n_jobs=4)]: Done 458 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 2636 tasks      | elapsed:   12.1s
[Parallel(n_jobs=4)]: Done 6290 tasks      | elapsed:   32.4s
[Parallel(n_jobs=4)]: Done 7875 out of 7875 | elapsed:   46.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


completed
dt
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


completed
sv
Fitting 5 folds for each of 8 candidates, totalling 40 fits
completed


[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    0.7s finished


In [50]:
results_df

,classifier_name,classifier,best_params,ROC_AUC
2,xg,"XGBClassifier(alpha=10, base_score=0.5, booste...","{'colsample_bytree': 0.7, 'gamma': 0.1, 'max_d...",0.961226
1,knn,KNeighborsClassifier(n_neighbors=4),{'n_neighbors': 4},0.945093
3,dsc,"DecisionTreeClassifier(max_depth=7, random_sta...",{'max_depth': 7},0.877355
0,logreg,"LogisticRegression(C=1000.0, random_state=88)","{'C': 1000.0, 'penalty': 'l2'}",0.848441
4,svm_rbf,"SVC(C=1000, gamma=0.001, random_state=88)","{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}",0.838545


In [66]:
file=['Recession_in_6mo.xg','Recession_in_12mo.xg','Recession_in_24mo.xg']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(X_test.iloc[:,1:7])[:,1])
    

[0.2815763  0.1865972  0.27440196 0.7308532  0.2938432  0.20749871
 0.27970266 0.47188875 0.18845406 0.3822774  0.2846068  0.18769956
 0.18618414 0.18618414 0.19192691 0.39065564 0.38137427 0.1944742
 0.583445   0.20254518 0.1865972  0.3474602  0.19528492 0.31137425
 0.79164267 0.39560944 0.45920232 0.19635983 0.59980005 0.2291414
 0.18338597 0.1865972  0.36874384 0.35525975 0.18618414 0.2447553
 0.73251426 0.18618414 0.22975495 0.18338597 0.4395739  0.36189508
 0.2915527  0.7664549  0.6887544  0.20659713 0.1865972  0.38438597
 0.23462771 0.5531914  0.76658875 0.37077552 0.1865972  0.20320515
 0.18338597 0.18188807 0.6347019  0.5407861  0.20494576 0.18462387
 0.38175446 0.22901417 0.26329216 0.3474602  0.23542735 0.6393629
 0.6744209  0.59772843 0.22411445 0.73404247 0.69216716 0.26422647
 0.1865972  0.19514693 0.23059994 0.28491643 0.1906456  0.20236544
 0.25726998 0.3317344  0.20772098 0.18544334 0.26611808 0.21434255
 0.34087873 0.2229151  0.18618414 0.20749082 0.19974504 0.7291121


array([0.73615223, 0.19107905, 0.6011711 , 0.24393572, 0.18949933,
       0.18258965, 0.72270447, 0.18946934, 0.72454816, 0.23888355,
       0.68428206, 0.18890615, 0.22985229, 0.748349  , 0.19156268,
       0.20030676, 0.18949933, 0.2704547 , 0.25026807, 0.28545862,
       0.25615078, 0.22681324, 0.2940762 , 0.20661014, 0.28718585,
       0.18258965, 0.7767031 , 0.52240855, 0.18755472, 0.6049654 ,
       0.24105248, 0.55216056, 0.23651756, 0.51314044, 0.18403418,
       0.19098578, 0.18533996, 0.22404079, 0.42218754, 0.48907316,
       0.19153246, 0.18744291, 0.2146066 , 0.617962  , 0.24891178,
       0.20276694, 0.21607898, 0.23156068, 0.20667568, 0.5141725 ,
       0.7195315 , 0.32937303, 0.20886786, 0.202452  , 0.73598593,
       0.27812782, 0.47232386, 0.23629573, 0.24455641, 0.18599334,
       0.74683404, 0.2062959 , 0.48594388, 0.20014969, 0.4793051 ,
       0.20645887, 0.43131942, 0.27383536, 0.73800594, 0.19107905,
       0.1982789 , 0.25921527, 0.18776146, 0.22877754, 0.27157

In [56]:
xg = results[2].get('classifier').predict_proba(X_train.iloc[:,1:7])[:,1]

In [57]:
xg

array([0.28793624, 0.18292074, 0.27011594, 0.18787041, 0.31073636,
       0.20828253, 0.22055402, 0.44974896, 0.369951  , 0.23782492,
       0.3407697 , 0.27529347, 0.19720933, 0.7288237 , 0.2207299 ,
       0.67684454, 0.37672618, 0.21137287, 0.40129402, 0.26387745,
       0.32328925, 0.73750305, 0.2907698 , 0.3267297 , 0.3839437 ,
       0.19102523, 0.24509059, 0.7399044 , 0.2502141 , 0.7344686 ,
       0.24342312, 0.34887207, 0.38972926, 0.37718645, 0.20542584,
       0.1953372 , 0.27480367, 0.21137287, 0.73687   , 0.30137634,
       0.235625  , 0.24348897, 0.1848792 , 0.3359445 , 0.19223703,
       0.254544  , 0.7530061 , 0.35605177, 0.2010681 , 0.7206985 ,
       0.3993811 , 0.23102468, 0.23635472, 0.23272485, 0.3061575 ,
       0.22183679, 0.3597457 , 0.25035444, 0.25116014, 0.29978907,
       0.7270807 , 0.27529347, 0.7471374 , 0.27240103, 0.38299775,
       0.26488283, 0.22685255, 0.20193392, 0.20899321, 0.18292074,
       0.29489386, 0.70226765, 0.20152095, 0.43089408, 0.74864

In [205]:

param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf'],'probability':[True],'class_weight':['balanced'],'tol':[1e-3]}

In [208]:
X_train['Recession_in_6mo'].value_counts()


0    515
1     88
Name: Recession_in_6mo, dtype: int64

In [ ]:
r\

In [195]:
from sklearn.model_selection import GridSearchCV  
for i in output_feature:
    grid = GridSearchCV(svm.SVC(),param_grid,verbose=2,scoring='recall',cv=10)
    grid.fit(X_train.iloc[:,1:7],y_train[i])
    svmc=grid.best_estimator_
    predicted_probability[i+"_probability"] = svmc.predict_proba(x_testing_data.iloc[:,1:7])[:,1]
    past_predict[i+"_probability"]=svmc.predict_proba(X_train.iloc[:,1:7])[:,1]
    test_prob[i+"_probability"]=svmc.predict_proba(X_test.iloc[:,1:7])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      
    



Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.1s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=Tr

[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, 

[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=r

...

[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=0.1, class_weight=balanced, gamma=1, kernel=rbf, probability=True, 

[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=1, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, 

[CV]  C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.1, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=10, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=10, class_weight=balanced, gamma=0.01, kernel=rbf, pro

[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001, total=   0.0s
[CV] C=100, class_weight=balanced, gamma=0.01, kernel=rbf, probability=True, tol=0.001 
[CV]  C=100, class_weight=balanced, gamma=0.0

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    6.9s finished
/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [165]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:7], y_train[i], cv=10, scoring='recall')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:7], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:7])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:7])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:7])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:7])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      


Parameter            Average accuracy         
(1, 0.001)	       0.7535714285714286

Parameter            Average accuracy         
(1, 0.0001)	                      0.0

Parameter            Average accuracy         
(1, 0.04)	       0.8232142857142856

Parameter            Average accuracy         
(10, 0.001)	                    0.825

Parameter            Average accuracy         
(10, 0.0001)	       0.7535714285714286

Parameter            Average accuracy         
(10, 0.04)	       0.9053571428571429

Parameter            Average accuracy         
(100, 0.001)	       0.9053571428571429

Parameter            Average accuracy         
(100, 0.0001)	                    0.825

Parameter            Average accuracy         
(100, 0.04)	       0.8910714285714285

Parameter            Average accuracy         
(1000, 0.001)	       0.8892857142857142

Parameter            Average accuracy         
(1000, 0.0001)	       0.9053571428571429

Parameter            Average accuracy         
(

In [193]:

predicted_probability['Date']= x_testing_data['Date'].tolist()

In [194]:
predicted_probability

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.122807,0.151529,0.195708,2018-04-01
1,0.122823,0.150069,0.209471,2018-05-01
2,0.122640,0.166263,0.343646,2018-06-01
3,0.122517,0.177419,0.369595,2018-07-01
4,0.122404,0.191447,0.489910,2018-08-01
5,0.122412,0.191811,0.470761,2018-09-01
6,0.122439,0.192621,0.262753,2018-10-01
7,0.122260,0.215541,0.346505,2018-11-01
8,0.121700,0.288218,0.209712,2018-12-01
9,0.121514,0.316645,0.282517,2019-01-01


In [159]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()


In [160]:
past_predict.append(test_prob)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.091898,0.218020,0.226785,1963-11-01
1,0.183318,0.199331,0.089594,2007-10-01
2,0.039110,0.057145,0.445061,1978-04-01
3,0.025278,0.032627,0.173658,1984-08-01
4,0.092504,0.214959,0.219774,1963-12-01
...,...,...,...,...
221,0.004021,0.005046,0.000293,1993-01-01
222,0.168893,0.342868,0.366345,1968-06-01
223,0.017738,0.064195,0.237720,1961-10-01
224,0.330863,0.351005,0.221029,1978-11-01


In [161]:
past_predict.append(predicted_probability)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.091898,0.218020,2.267852e-01,1963-11-01
1,0.183318,0.199331,8.959351e-02,2007-10-01
2,0.039110,0.057145,4.450611e-01,1978-04-01
3,0.025278,0.032627,1.736585e-01,1984-08-01
4,0.092504,0.214959,2.197743e-01,1963-12-01
...,...,...,...,...
22,0.091670,0.255141,1.724135e-02,2019-12-01
23,0.181265,0.273751,1.274143e-02,2020-01-01
24,0.358971,0.347465,4.872360e-03,2020-02-01
25,0.675139,0.176969,4.610745e-05,2020-03-01


In [63]:
file=['Recession_in_6mo.xg','Recession_in_12mo.xg']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(x[:,1:7])[:,1])
    
    
    



NameError: name 'test' is not defined

In [131]:
test=recession_data[recession_data['Date'] == '2007-02-01']

test

,Date,Payrolls_3mo_vs_12mo,Effective_Fed_Funds_12_chg,CPI_All_Items_3_mo_annualised,10Y_Treasury_Rate_12_chg,3M_10Y_Treasury_Spread,S&P_500_Index_12_chg,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
156,2007-02-01,1.001109,1.171492,1.044813,1.032823,-0.31,1.098512,0,1,1


In [43]:
X_train,X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2000, random_state=0,class_weight='balanced')
clf.fit(X_train.iloc[:,1:7], y_train)

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=2000, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [55]:
y_predict = clf.predict_proba(X_test.iloc[:,1:7])

  

In [56]:
y_predict

[array([[0.9, 0.1],
        [0.9, 0.1],
        [1. , 0. ],
        [0.9, 0.1],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.7, 0.3],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [0.9, 0.1],
        [0.9, 0.1],
        [0.9, 0.1],
        [0.6, 0.4],
        [0.9, 0.1],
        [1. , 0. ],
        [0.7, 0.3],
        [0.4, 0.6],
        [1. , 0. ],
        [0.5, 0.5],
        [0.6, 0.4],
        [1. , 0. ],
        [0.8, 0.2],
        [1. , 0. ],
        [1. , 0. ],
        [0.5, 0.5],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        [0.2, 0.8],
        [1. , 0. ],
        [0.6, 0.4],
        [1. , 0. ],
        [0.8, 0.2],
        [1. , 0. ],
        [0.9, 0.1],
        [1. , 0. ],
        [1. , 0. ],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.8, 0.2],
        [0.8, 0.2],
        [1. , 0. ],
        [0.5, 0.5],
        [0.8, 0.2],


In [102]:
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/website/house/sydney_new_data.csv')

In [103]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [104]:
df.iloc[1:1000,:]

,SUBURB,ADDRESS,bedroom_count,Type,dwelling_type,sold_price,Method,agency_name,Date Sold,postcode,Population,Distance from CBD,Hospital,School,Super Market,latitude,longitude
1,ABBOTSFORD,"6/431 Great North Road, Abbotsford",2,u,Apartment / Unit / Flat,735000.0,Auction,PRB Real Estate,14/05/2018,2046,5377,7.4,0,1,0,-33.854317,151.129654
2,ALEXANDRIA,"64/57 Ralph Street, Alexandria",2,u,Apartment / Unit / Flat,785000.0,Private Treaty,The Agency Eastern Suburbs,14/05/2018,2015,8264,3.5,0,3,0,-33.919315,151.197900
3,ALEXANDRIA,"64/57 Ralph Street, Alexandria",2,u,Apartment / Unit / Flat,785000.0,Private Treaty,The Agency Eastern Suburbs,14/05/2018,2015,8264,3.5,0,3,0,-33.919315,151.197900
4,ASHFIELD,"208/425 Liverpool Road, Ashfield",2,u,Apartment / Unit / Flat,795000.0,Private Treaty,McGrath Leichhardt,14/05/2018,2131,23839,7.9,0,6,0,-33.887860,151.118118
5,BEACON HILL,"56 Brooker Avenue, Beacon Hill",3,h,House,1200000.0,Private Treaty,One Agency Hazlett & Poole,14/05/2018,2100,7456,13.7,0,1,0,-33.747536,151.254990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NEWTOWN,"32/20 Fitzgerald Street, Newtown",3,u,Apartment / Unit / Flat,1010000.0,Auction,Ray White Newtown,24/05/2018,2042,15025,4.1,0,6,0,-33.891884,151.180700
996,NEWTOWN,"34 Darley Street, Newtown",3,h,Terrace,1370000.0,Private Treaty,Ray White Petersham,24/05/2018,2042,15025,4.1,0,6,0,-33.906540,151.179520
997,NORTH NARRABEEN,"24 Powderworks Road, North Narrabeen",3,h,House,1400000.0,Private Treaty,Stone Real Estate Mona Vale,24/05/2018,2101,5854,20.0,0,3,0,-33.702910,151.293300
998,PADDINGTON,"69 Harris Street, Paddington",3,h,House,4375000.0,Auction,McGrath Double Bay,24/05/2018,2021,12909,2.7,0,4,0,-33.884480,151.236786


In [83]:
df.to_csv('/Users/sebin/Desktop/companyandngo/website/melboure_house.csv')

In [105]:
df.columns

Index(['SUBURB', 'ADDRESS', 'bedroom_count', 'Type', 'dwelling_type',
       'sold_price', 'Method', 'agency_name', 'Date Sold', 'postcode',
       'Population', 'Distance from CBD', 'Hospital', 'School', 'Super Market',
       'latitude', 'longitude'],
      dtype='object')

In [109]:
##logistic regression

In [106]:
df.iloc[1:1000,:].to_csv('/Users/sebin/Desktop/companyandngo/website/1000_sydney.csv')

In [110]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4,.04]
features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']



In [112]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [107]:

# implementing a Logistic Regression model with random hyperparameters
LR = LogisticRegression(penalty='l1',dual=False,max_iter=110, solver='liblinear')
LR.fit(X_train,Y_train)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=110, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


# In[158]:


#setting the parameters
dual=[True,False]
max_iter=[100,110,120,130,140]
C = [1.0,1.5,2.0,2.5,3.0,3.5,4.0]
param_grid = dict(dual=dual,max_iter=max_iter,C=C)


# In[159]:


#implementing a Grid search on our defined hyperparameters
LR = LogisticRegression(penalty='l2')
G_search = GridSearchCV(estimator=LR, param_grid=param_grid, cv = 3, n_jobs=-1)
import time

starting_time = time.time()
G_search_results = G_search.fit(X_train,Y_train)
# Summarizing the  results
print("Best accuracy: %f gained by using this set of parameters%s" % (G_search_results.best_score_, G_search_results.best_params_))
print("time of execution: " + str((time.time() - starting_time)) + ' ms')


# In[160]:


#implementing a Random search on our defined hyperparameters
R_search = RandomizedSearchCV(estimator=LR, param_distributions=param_grid, cv = 3, n_jobs=-1)
starting_time = time.time()
R_search_results = R_search.fit(X_train,Y_train)
# Summarizing the  results
print("Best accuracy: %f gained by using this set of parameters%s" % (R_search_results.best_score_, R_search_results.best_params_))
print("time of execution: " + str((time.time() - starting_time)) + ' ms')


FileNotFoundError: [Errno 2] File b'final_features.csv' does not exist: b'final_features.csv'